In [1]:
from avalanche.benchmarks import SplitMNIST
import lovely_tensors as lt

In [15]:
seed = 42

benchmark = SplitMNIST(n_experiences=5,return_task_id=True, shuffle=False,)

In [16]:
dataset = benchmark.train_stream[2].dataset
lt.lovely(dataset[0][0])

tensor[1, 28, 28] n=784 x∈[-0.424, 2.821] μ=0.023 σ=1.014

In [19]:
dataset.return_task_id

AttributeError: 'AvalancheSubset' object has no attribute 'return_task_id'